# Importing libraries

In [189]:
import neo4j
import psycopg2
import csv
import math
import numpy as np
import pandas as pd
import os
import sys
from operator import itemgetter
import helper_functions
from helper_functions import postgress_helper
from helper_functions import neo4j_helper
from IPython.display import display

# Adding custom package path

In [190]:
from os.path import dirname
sys.path.append(dirname(os.getcwd() + '/helper_functions'))

# Instantiating helper classes

In [191]:
pg_helper = postgress_helper.postgress_helper(db_name = "northwinds_db")
n4j_helper = neo4j_helper.neo4j_helper(userid = "neo4j", password = "w205", db_name = "neo4j")

# Wiping out databases in neo4j

In [192]:
n4j_helper.neo4j_wipe_out_database()
n4j_helper.neo4j_number_nodes_relationships()

-------------------------
  Nodes: 0
  Relationships: 0
-------------------------


# Creating cutomers nodes

In [193]:
query = """
SELECT     cust.*
         , ord.total_orders
FROM       customers cust
INNER JOIN (SELECT customer_id
                 , COUNT(order_id) AS total_orders 
            FROM   orders
            GROUP BY customer_id) ord
ON         cust.customer_id = ord.customer_id
;
"""
customer_df = pg_helper.select_query_pandas(query)
for cust in customer_df.values.tolist():
    cust_id, cust_name = 'cust_' + cust[0], cust[1]
    total_orders = cust[-1]
    label_name    = 'customer'
    label_value   = cust_id
    label_details = f"{label_value}:{label_name}"
    node_property_string = '{customer_id: "' + cust_id + '", customer_name: "' + cust_name + '", total_orders: ' + str(total_orders) + '}'
    n4j_helper.neo4j_create_node(label_details   = label_details
                               , node_property   = node_property_string)
n4j_helper.neo4j_number_nodes_relationships()


-------------------------
  Nodes: 89
  Relationships: 0
-------------------------


# Creating suppliers nodes

In [194]:
query = """
SELECT *
FROM   suppliers
;
"""
supplier_df = pg_helper.select_query_pandas(query)
for suppl in supplier_df.values.tolist():
    supplier_id, supplier_name = 'suppl_' + str(suppl[0]), suppl[1]
    label_name    = 'supplier'
    label_value   = supplier_id
    label_details = f"{label_value}:{label_name}"
    node_property_string = '{supplier_id: "' + supplier_id + '", customer_name: "' + supplier_name + '"}'
    n4j_helper.neo4j_create_node(label_details   = label_details
                               , node_property   = node_property_string)
n4j_helper.neo4j_number_nodes_relationships()

-------------------------
  Nodes: 118
  Relationships: 0
-------------------------


# Creating categories nodes

In [195]:
query = """
SELECT *
FROM   categories
;
"""
ctg_df = pg_helper.select_query_pandas(query)
for ctg in ctg_df.values.tolist():
    ctg_id = 'ctg_' + str(ctg[0])
    ctg_name = ctg[1]
    label_name    = 'ctg'
    label_value   = ctg_id
    label_details = f"{label_value}:{label_name}"
    node_property_string = '{ctg_id: "' + ctg_id + '", ctg_name: "' + ctg_name + '"}'
    n4j_helper.neo4j_create_node(label_details   = label_details
                               , node_property   = node_property_string)
n4j_helper.neo4j_number_nodes_relationships()

-------------------------
  Nodes: 126
  Relationships: 0
-------------------------


# Creating products nodes

In [196]:
query = """
SELECT *
FROM   products
WHERE  discontinued = 0
;
"""
prod_df = pg_helper.select_query_pandas(query)
for prod in prod_df.values.tolist():
    prod_id = 'prod_' + str(prod[0])
    prod_name = prod[1]
    supplier_id = 'suppl_' + str(prod[2])
    ctg_id =  'ctg_' + str(prod[3])
    label_name    = 'product'
    label_value   = prod_id
    label_details = f"{label_value}:{label_name}"
    node_property_string = '{prod_id: "' + prod_id + '", prod_name: "' + prod_name + '", supplier_id: "' + supplier_id + '", ctg_id: "' + ctg_id + '"}'
    n4j_helper.neo4j_create_node(label_details   = label_details
                               , node_property   = node_property_string)
n4j_helper.neo4j_number_nodes_relationships()

-------------------------
  Nodes: 193
  Relationships: 0
-------------------------


# Creating orders nodes

In [197]:
query = """
SELECT *
FROM   orders
;
"""
order_header_df = pg_helper.select_query_pandas(query)
for ord_hdr in order_header_df.values.tolist():
    order_id = 'ord_' + str(ord_hdr[0])
    customer_id = 'cust_' + ord_hdr[1]
    label_name    = 'order_header'
    label_value   = order_id
    label_details = f"{label_value}:{label_name}"
    node_property_string = '{order_id :"' + order_id + '", customer_id: "' + customer_id + '"}'
    n4j_helper.neo4j_create_node(label_details = label_details
                               , node_property = node_property_string)
n4j_helper.neo4j_number_nodes_relationships()

-------------------------
  Nodes: 1023
  Relationships: 0
-------------------------


# Creating order details nodes and customer product relationship thru rating

In [198]:
query = """
SELECT     ord_dtl.*
         , dtl_agg.total_products_purchased
         , ord_hdr.customer_id
         , cust_agg.total_orders
         , cust_tot_ord_qty.total_purchased
FROM       order_details ord_dtl
INNER JOIN orders        ord_hdr
ON         ord_hdr.order_id = ord_dtl.order_id
INNER JOIN customers cust
ON         ord_hdr.customer_id = cust.customer_id
INNER JOIN (SELECT customer_id
                 , COUNT(order_id) AS total_orders 
            FROM   orders
            GROUP BY customer_id
           ) cust_agg
ON         cust_agg.customer_id = ord_hdr.customer_id
INNER JOIN (SELECT     cust.customer_id
                     , dtl.product_id
                     , SUM(dtl.quantity) AS total_products_purchased
            FROM       customers     cust
            INNER JOIN orders        hdr
            ON         cust.customer_id = hdr.customer_id
            INNER JOIN order_details dtl
            ON         hdr.order_id = dtl.order_id
            GROUP BY   cust.customer_id
                     , dtl.product_id
           ) dtl_agg
ON         dtl_agg.customer_id = cust.customer_id
AND        dtl_agg.product_id  = ord_dtl.product_id
INNER JOIN (SELECT     cust.customer_id
                     , SUM(dtl.quantity) AS total_purchased
            FROM       customers     cust
            INNER JOIN orders        hdr
            ON         cust.customer_id = hdr.customer_id
            INNER JOIN order_details dtl
            ON         hdr.order_id = dtl.order_id
            GROUP BY   cust.customer_id
           ) cust_tot_ord_qty
ON         cust_tot_ord_qty.customer_id = cust.customer_id
;
"""
order_line_df = pg_helper.select_query_pandas(query)
order_line_df['rating'] = order_line_df['total_products_purchased'] / order_line_df['total_purchased']
for ord_line in order_line_df.values.tolist():
    order_id = 'ord_' + str(ord_line[0])
    prod_id = 'prod_' + str(ord_line[1])
    ord_qty = ord_line[3]
    
    total_products_purchased = ord_line[-5]
    customer_id = 'cust_' + ord_line[-4]
    total_orders = ord_line[-3]
    total_purchased = ord_line[-2]
    rating = ord_line[-1]
    
    label_name    = 'order_line'
    label_value   = order_id
    label_details = f"{label_value}:{label_name}"
    misc_str = f", customer_id: '{customer_id}', total_orders: {total_orders}, total_products_purchased: {total_products_purchased}, total_purchased: {total_purchased}, rating: {rating}"
    node_property_string = '{order_id :"' + order_id + '", prod_id: "' + prod_id + '", qty: ' + str(ord_qty) + misc_str + "}"
    n4j_helper.neo4j_create_node(label_details = label_details
                               , node_property = node_property_string)
    
    rating_string = "{" + f"rating : {rating}" + "}"
    search_string = f" AND c.customer_id = '{customer_id}' AND d.prod_id = '{prod_id}'"
    query = f"""MATCH (c:customer),(d:order_line),(p:product)
                WHERE    c.customer_id = d.customer_id
                AND      d.prod_id = p.prod_id {search_string}
                MERGE (c)-[:rated {rating_string}]->(p)"""
    #print(query)
    n4j_helper.neo4j_session.run(query)
    
n4j_helper.neo4j_number_nodes_relationships()

-------------------------
  Nodes: 3178
  Relationships: 1455
-------------------------


# Creating relationship between product and categories
# category<-[:part_of]-product

In [199]:
query = """MATCH (p:product),(c:ctg)
WHERE c.ctg_id = p.ctg_id
MERGE (p)-[:part_of]->(c)"""
n4j_helper.neo4j_session.run(query)
n4j_helper.neo4j_number_nodes_relationships()

-------------------------
  Nodes: 3178
  Relationships: 1522
-------------------------


# Creating relationship between product and suppliers
# supplier-[:supplies]->product

In [200]:
query = """MATCH (p:product),(s:supplier)
WHERE s.supplier_id = p.supplier_id
MERGE (s)-[:supplies]->(p)"""
n4j_helper.neo4j_session.run(query)
n4j_helper.neo4j_number_nodes_relationships()

-------------------------
  Nodes: 3178
  Relationships: 1589
-------------------------


# Creating relationship between customer and order
# customer-[:orders]->order_header

In [201]:
query = """MATCH (o:order_header),(c:customer)
WHERE c.customer_id = o.customer_id
MERGE (c)-[:orders]->(o)"""
n4j_helper.neo4j_session.run(query)
n4j_helper.neo4j_number_nodes_relationships()

-------------------------
  Nodes: 3178
  Relationships: 2419
-------------------------


# Creating relationship between order header and order line
# order_header-[:comprised_of]->order_line

In [202]:
query = """MATCH (o:order_header),(d:order_line)
WHERE o.order_id = d.order_id
MERGE (o)-[:comprised_of]->(d)"""
n4j_helper.neo4j_session.run(query)
n4j_helper.neo4j_number_nodes_relationships()

-------------------------
  Nodes: 3178
  Relationships: 4574
-------------------------


# Creating relationship between order line and product
# order_line-[:contains]->product

In [203]:
query = """MATCH (p:product),(d:order_line)
WHERE d.prod_id = p.prod_id
MERGE (d)-[:contains]->(p)"""
n4j_helper.neo4j_session.run(query)
n4j_helper.neo4j_number_nodes_relationships()

-------------------------
  Nodes: 3178
  Relationships: 6419
-------------------------


# List down total quantity of products purcased by customers

In [204]:
query = '''MATCH (cust:customer)-[:orders]->(oh:order_header)-[:comprised_of]->(od:order_line)-[:contains]->(p:product),
      (p)-[:part_of]->(c:ctg)
RETURN  DISTINCT cust.customer_name AS customer_name, SUM(od.qty) AS total_product_qty_purchased
ORDER BY total_product_qty_purchased DESC
'''
n4j_helper.neo4j_run_query_pandas(query)

,customer_name,total_product_qty_purchased
0,Save-a-lot Markets,3963
1,Ernst Handel,3835
2,QUICK-Stop,3462
3,Frankenversand,1434
4,Hungry Owl All-Night Grocers,1377
...,...,...
84,The Cracker Box,39
85,GROSELLA-Restaurante,24
86,North/South,22
87,Lazy K Kountry Store,20


# List down total quantity of produce purcased by customers

In [205]:
query = '''MATCH (cust:customer)-[:orders]->(oh:order_header)-[:comprised_of]->(od:order_line)-[:contains]->(p:product),
      (p)-[:part_of]->(c:ctg {ctg_name:"Produce"})
RETURN  DISTINCT cust.customer_name AS customer_name, SUM(od.qty) AS total_produce_qty_purchased
ORDER BY total_produce_qty_purchased DESC
'''
n4j_helper.neo4j_run_query_pandas(query)

,customer_name,total_produce_qty_purchased
0,QUICK-Stop,242
1,Ernst Handel,235
2,Folk och fä HB,148
3,Save-a-lot Markets,137
4,Hungry Owl All-Night Grocers,121
5,Bon app',118
6,Eastern Connection,100
7,Ottilies Käseladen,85
8,Rattlesnake Canyon Grocery,77
9,Chop-suey Chinese,71


# List down each produce purchased along with quantity for each customer

In [206]:
query = '''MATCH (cust:customer)-[:orders]->(oh:order_header)-[:comprised_of]->(od:order_line)-[:contains]->(p:product),
      (p)-[:part_of]->(c:ctg {ctg_name:"Produce"})
RETURN  DISTINCT cust.customer_name AS customer_name, p.prod_name AS prod_name, SUM(od.qty) AS total_produce_qty_purchased
ORDER BY total_produce_qty_purchased DESC
'''
n4j_helper.neo4j_run_query_pandas(query)

,customer_name,prod_name,total_produce_qty_purchased
0,QUICK-Stop,Uncle Bob's Organic Dried Pears,135
1,Ernst Handel,Manjimup Dried Apples,120
2,Eastern Connection,Uncle Bob's Organic Dried Pears,100
3,QUICK-Stop,Manjimup Dried Apples,92
4,Folk och fä HB,Manjimup Dried Apples,88
...,...,...,...
76,Ana Trujillo Emparedados y helados,Tofu,3
77,Great Lakes Food Market,Tofu,3
78,Que Delícia,Manjimup Dried Apples,2
79,Wolski Zajazd,Manjimup Dried Apples,2


# Lists down products bought by each customer, that is total number of orders aong with total quantities ordered

In [207]:
query = '''MATCH (cust:customer)-[:orders]->(oh:order_header)-[:comprised_of]->(od:order_line)-[:contains]->(p:product),
      (p)-[:part_of]->(c:ctg)
RETURN  
DISTINCT cust.customer_name AS customer_name
       , c.ctg_name         AS category_name
       , p.prod_name        AS prod_name
       , count(oh.order_id) AS total_orders
       , sum(od.qty)        AS total_prod_qty_purchased
ORDER BY total_prod_qty_purchased DESC
'''
n4j_helper.neo4j_run_query_pandas(query)

,customer_name,category_name,prod_name,total_orders,total_prod_qty_purchased
0,Save-a-lot Markets,Seafood,Konbu,4,248
1,QUICK-Stop,Dairy Products,Camembert Pierrot,4,243
2,Save-a-lot Markets,Confections,Scottish Longbreads,4,220
3,Ernst Handel,Grains/Cereals,Wimmers gute Semmelknödel,4,199
4,Save-a-lot Markets,Beverages,Rhönbräu Klosterbier,3,198
...,...,...,...,...,...
1450,Rattlesnake Canyon Grocery,Produce,Tofu,1,1
1451,Centro comercial Moctezuma,Seafood,Gravad lax,1,1
1452,Consolidated Holdings,Seafood,Konbu,1,1
1453,Franchi S.p.A.,Seafood,Boston Crab Meat,1,1


# Content based recommendation where we are recommending similar products (same category) to the customers what other customers have also bought


In [208]:
query = """MATCH (cust1:customer)-[:rated]->(p:product)
MATCH (p)<-[:rated]-(cust2:customer)
MATCH (cust2)-[:rated]->(p2:product)
MATCH (p2)-[:part_of]->(cat:ctg)
MATCH (p)-[:part_of]->(cat)
WHERE  NOT EXISTS ((cust1)-[:rated]->(p2))
RETURN 
DISTINCT cust1.customer_name          AS customer_name
       , COLLECT(p2.prod_name)        AS recommended_product_list
ORDER BY customer_name"""
n4j_helper.neo4j_run_query_pandas(query)

,customer_name,recommended_product_list
0,Alfreds Futterkiste,"[Chef Anton's Cajun Seasoning, Northwoods Cran..."
1,Ana Trujillo Emparedados y helados,"[Flotemysost, Queso Manchego La Pastora, Racle..."
2,Antonio Moreno Taquería,"[Flotemysost, Mozzarella di Giovanni, Queso Ma..."
3,Around the Horn,"[Ikura, Carnarvon Tigers, Nord-Ost Matjesherin..."
4,B's Beverages,"[Chef Anton's Cajun Seasoning, Grandma's Boyse..."
...,...,...
84,Wartian Herkku,"[Aniseed Syrup, Grandma's Boysenberry Spread, ..."
85,Wellington Importadora,"[Ikura, Jack's New England Clam Chowder, Nord-..."
86,White Clover Markets,"[Aniseed Syrup, Grandma's Boysenberry Spread, ..."
87,Wilman Kala,"[Aniseed Syrup, Chef Anton's Cajun Seasoning, ..."


# Content based recommendation where we are recommending similar products (same category) to the customers that they have not bought yet, but they have previously purchased other products from the same category

In [209]:
query = """MATCH (cust1:customer)-[:rated]->(p:product)
MATCH (p2)-[:part_of]->(cat:ctg)
MATCH (p)-[:part_of]->(cat)
WHERE  NOT EXISTS ((cust1)-[:rated]->(p2))
RETURN 
DISTINCT cust1.customer_name          AS customer_name
       , COLLECT(p2.prod_name)        AS recommended_product_list
ORDER BY customer_name"""
n4j_helper.neo4j_run_query_pandas(query)

,customer_name,recommended_product_list
0,Alfreds Futterkiste,"[Côte de Blaye, Ipoh Coffee, Sasquatch Ale, St..."
1,Ana Trujillo Emparedados y helados,"[Côte de Blaye, Lakkalikööri, Ipoh Coffee, Sas..."
2,Antonio Moreno Taquería,"[Côte de Blaye, Lakkalikööri, Steeleye Stout, ..."
3,Around the Horn,"[Côte de Blaye, Lakkalikööri, Ipoh Coffee, Sas..."
4,B's Beverages,"[Côte de Blaye, Lakkalikööri, Sasquatch Ale, R..."
...,...,...
84,Wartian Herkku,"[Côte de Blaye, Lakkalikööri, Sasquatch Ale, R..."
85,Wellington Importadora,"[Côte de Blaye, Lakkalikööri, Sasquatch Ale, S..."
86,White Clover Markets,"[Lakkalikööri, Ipoh Coffee, Chartreuse verte, ..."
87,Wilman Kala,"[Côte de Blaye, Sasquatch Ale, Steeleye Stout,..."


# Showing customer's product ratings

In [210]:
query = """MATCH (me:customer)-[r:rated]->(p:product)
RETURN 
DISTINCT me.customer_name AS customer_name
       , p.prod_name      AS product_name
       , r.rating         AS rating
ORDER BY rating DESC"""
n4j_helper.neo4j_run_query_pandas(query)

,customer_name,product_name,rating
0,Centro comercial Moctezuma,Sir Rodney's Scones,0.909091
1,Lazy K Kountry Store,Queso Cabrales,0.500000
2,Lazy K Kountry Store,Boston Crab Meat,0.500000
3,Laughing Bacchus Wine Cellars,Jack's New England Clam Chowder,0.322581
4,North/South,Outback Lager,0.300000
...,...,...,...
1450,Ernst Handel,Mascarpone Fabioli,0.001321
1451,Rattlesnake Canyon Grocery,Grandma's Boysenberry Spread,0.000723
1452,Rattlesnake Canyon Grocery,Ikura,0.000723
1453,Rattlesnake Canyon Grocery,Tofu,0.000723


# List down product rating difference for a particular customer

In [211]:
query="""MATCH (c1:customer)-[r1:rated]->(p:product)<-[r2:rated]-(c2:customer)
WHERE c1.customer_id = 'cust_CENTC'
RETURN c1.customer_id AS customer_1
     , c2.customer_id AS customer_2
     , p.prod_name AS product_rated
     , r1.rating   AS rating_given_by_customer_1
     , r2.rating   AS rating_given_by_customer_2
     , ABS(r1.rating - r2.rating) AS rating_difference
ORDER BY rating_difference DESC
"""
n4j_helper.neo4j_run_query_pandas(query)

,customer_1,customer_2,product_rated,rating_given_by_customer_1,rating_given_by_customer_2,rating_difference
0,cust_CENTC,cust_WARTH,Sir Rodney's Scones,0.909091,0.013569,0.895522
1,cust_CENTC,cust_QUICK,Sir Rodney's Scones,0.909091,0.015148,0.893943
2,cust_CENTC,cust_GODOS,Sir Rodney's Scones,0.909091,0.015190,0.893901
3,cust_CENTC,cust_OLDWO,Sir Rodney's Scones,0.909091,0.024876,0.884215
4,cust_CENTC,cust_ERNSH,Sir Rodney's Scones,0.909091,0.026414,0.882677
5,cust_CENTC,cust_QUEEN,Sir Rodney's Scones,0.909091,0.029098,0.879993
6,cust_CENTC,cust_LINOD,Sir Rodney's Scones,0.909091,0.030928,0.878163
7,cust_CENTC,cust_SUPRD,Sir Rodney's Scones,0.909091,0.033582,0.875509
8,cust_CENTC,cust_MEREP,Sir Rodney's Scones,0.909091,0.043478,0.865613
9,cust_CENTC,cust_KOENE,Sir Rodney's Scones,0.909091,0.044297,0.864794


In [212]:
query="""MATCH (c1:customer)-[r1:rated]->(p:product)<-[r2:rated]-(c2:customer)
RETURN c1.customer_id AS customer_1
     , c2.customer_id AS customer_2
     , p.prod_name AS product_rated
     , r1.rating   AS rating_given_by_customer_1
     , r2.rating   AS rating_given_by_customer_2
     , ABS(r1.rating - r2.rating) AS rating_difference
ORDER BY rating_difference DESC
"""
df_rating_diff = n4j_helper.neo4j_run_query_pandas(query)
df_rating_diff

,customer_1,customer_2,product_rated,rating_given_by_customer_1,rating_given_by_customer_2,rating_difference
0,cust_WARTH,cust_CENTC,Sir Rodney's Scones,0.013569,0.909091,0.895522
1,cust_CENTC,cust_WARTH,Sir Rodney's Scones,0.909091,0.013569,0.895522
2,cust_QUICK,cust_CENTC,Sir Rodney's Scones,0.015148,0.909091,0.893943
3,cust_CENTC,cust_QUICK,Sir Rodney's Scones,0.909091,0.015148,0.893943
4,cust_GODOS,cust_CENTC,Sir Rodney's Scones,0.015190,0.909091,0.893901
...,...,...,...,...,...,...
35283,cust_BERGS,cust_SAVEA,Jack's New England Clam Chowder,0.039960,0.039935,0.000025
35284,cust_WARTH,cust_FOLIG,Raclette Courdavault,0.033921,0.033898,0.000023
35285,cust_FOLIG,cust_WARTH,Raclette Courdavault,0.033898,0.033921,0.000023
35286,cust_FRANS,cust_RICSU,Camembert Pierrot,0.074074,0.074074,0.000000


In [248]:
query="""MATCH (c1:customer)-[r1:rated]->(p:product)<-[r2:rated]-(c2:customer)
WHERE  r1.rating - r2.rating < .1 and r1.rating > .2 and r2.rating > .2
RETURN c1.customer_name AS customer1, 
       c2.customer_name AS customer2, 
       p.prod_name AS product_name, 
       r1.rating AS rating_by_customer_1, 
       r2.rating AS rating_by_customer_2
ORDER BY r1.rating, r2.rating
"""
n4j_helper.neo4j_run_query_pandas(query)

,customer1,customer2,product_name,rating_by_customer_1,rating_by_customer_2
0,Antonio Moreno Taquería,Spécialités du monde,Queso Cabrales,0.206128,0.208333
1,Antonio Moreno Taquería,Océano Atlántico Ltda.,Queso Cabrales,0.206128,0.227273
2,Antonio Moreno Taquería,Lazy K Kountry Store,Queso Cabrales,0.206128,0.500000
3,Spécialités du monde,Antonio Moreno Taquería,Queso Cabrales,0.208333,0.206128
4,Spécialités du monde,Océano Atlántico Ltda.,Queso Cabrales,0.208333,0.227273
5,Spécialités du monde,Lazy K Kountry Store,Queso Cabrales,0.208333,0.500000
6,Rancho grande,Lazy K Kountry Store,Boston Crab Meat,0.217391,0.500000
7,Let's Stop N Shop,Centro comercial Moctezuma,Sir Rodney's Scones,0.220994,0.909091
8,Trail's Head Gourmet Provisioners,Bólido Comidas preparadas,Rhönbräu Klosterbier,0.224719,0.263158
9,Trail's Head Gourmet Provisioners,Spécialités du monde,Rhönbräu Klosterbier,0.224719,0.291667


In [249]:
cust_pair_dict = {}
cust_pair_prod_list = []
for e in df_rating_diff.values.tolist():
    #print('_'.join(sorted([e[0], e[1]])))
    if '_'.join(sorted([e[0], e[1]])) not in cust_pair_dict.keys() and abs(e[-2]-e[-3]) >= 0 and abs(e[-2]-e[-3]) < .01 and e[-2] > .2 and e[-3] > .2:
        cust_pair_prod_list.append([e[0], e[1], e[2]])
        cust_pair_dict['_'.join(sorted([e[0], e[1]]))] = 1
        


In [250]:
cust_pair_prod_list

[['cust_SPECD', 'cust_GROSR', 'Rhönbräu Klosterbier'],
 ['cust_SPECD', 'cust_ANTON', 'Queso Cabrales']]

In [251]:
query = """MATCH (c1:customer)-[r1:rated]->(p:product)<-[r2:rated]-(c2:customer)
WHERE   c1.customer_id ="cust_KOENE" and c2.customer_id ="cust_TRADH"
    and p.prod_name in ['Tarte au sucre','Pavlova']
RETURN c1, p, c2 , r1, r2 """
n4j_helper.neo4j_run_query_pandas(query)

,c1,p,c2,r1,r2
0,"(customer_name, customer_id, total_orders)","(supplier_id, ctg_id, prod_name, prod_id)","(customer_name, customer_id, total_orders)",(rating),(rating)


In [252]:
df_rating_diff[(df_rating_diff.customer_1 == "cust_KOENE") & (df_rating_diff.customer_2 == 'cust_TRADH') & (df_rating_diff.product_rated == 'Tarte au sucre')]

,customer_1,customer_2,product_rated,rating_given_by_customer_1,rating_given_by_customer_2,rating_difference
28760,cust_KOENE,cust_TRADH,Tarte au sucre,0.049834,0.039841,0.009993


# Creating similarity between customers

In [253]:
query = """MATCH (c1:customer)-[r1:rated]->(p:product)<-[r2:rated]-(c2:customer)
WITH
    SUM(r1.rating*r2.rating) as dot_product,
    SQRT( REDUCE(x=0.0, a IN COLLECT(r1.rating) | x + a^2) ) as r1_length,
    SQRT( REDUCE(y=0.0, b IN COLLECT(r2.rating) | y + b^2) ) as r2_length,
    c1,c2
MERGE (c1)-[s:similarity]-(c2)
SET s.similarity = dot_product / (r1_length * r2_length)
"""
n4j_helper.neo4j_run_query_pandas(query)
n4j_helper.neo4j_number_nodes_relationships()

-------------------------
  Nodes: 3245
  Relationships: 10033
-------------------------


In [254]:
query="""MATCH (c1:customer)-[r1:rated]->(p:product)<-[r2:rated]-(c2:customer)
WHERE p.prod_name = 'Original Frankfurter grüne Soße'
RETURN 
DISTINCT c1.customer_id AS customer_1
       , c2.customer_id AS customer_2
       , p.prod_name AS product_rated
       , r1.rating   AS rating_given_by_customer_1
       , r2.rating   AS rating_given_by_customer_2
       , ABS(r1.rating - r2.rating) AS rating_difference
ORDER BY rating_difference DESC
"""
n4j_helper.neo4j_run_query_pandas(query)

,customer_1,customer_2,product_rated,rating_given_by_customer_1,rating_given_by_customer_2,rating_difference
0,cust_FRANK,cust_TRAIH,Original Frankfurter grüne Soße,0.009836,0.202247,0.192411
1,cust_TRAIH,cust_FRANK,Original Frankfurter grüne Soße,0.202247,0.009836,0.192411
2,cust_LETSS,cust_TRAIH,Original Frankfurter grüne Soße,0.011050,0.202247,0.191197
3,cust_TRAIH,cust_LETSS,Original Frankfurter grüne Soße,0.202247,0.011050,0.191197
4,cust_ALFKI,cust_TRAIH,Original Frankfurter grüne Soße,0.011494,0.202247,0.190753
...,...,...,...,...,...,...
751,cust_LETSS,cust_ALFKI,Original Frankfurter grüne Soße,0.011050,0.011494,0.000445
752,cust_HANAR,cust_VAFFE,Original Frankfurter grüne Soße,0.083433,0.083333,0.000099
753,cust_VAFFE,cust_HANAR,Original Frankfurter grüne Soße,0.083333,0.083433,0.000099
754,cust_LEHMS,cust_QUICK,Original Frankfurter grüne Soße,0.031486,0.031558,0.000072


In [260]:
query="""MATCH (c1:customer)-[r1:rated]->(p:product)<-[r2:rated]-(c2:customer)
WHERE r1.rating > .2
AND   r2.rating > .2
AND   ABS(r1.rating - r2.rating) > .01 
AND   ABS(r1.rating - r2.rating) < .05
RETURN 
DISTINCT c1.customer_name AS customer_1_name
       , c2.customer_name AS customer_2_name
       , p.prod_name      AS product_rated
       , r1.rating        AS rating_given_by_customer_1
       , r2.rating        AS rating_given_by_customer_2
       , ABS(r1.rating - r2.rating) AS rating_difference
ORDER BY rating_difference DESC
"""
n4j_helper.neo4j_run_query_pandas(query)

,customer_1_name,customer_2_name,product_rated,rating_given_by_customer_1,rating_given_by_customer_2,rating_difference
0,La corne d'abondance,France restauration,Lakkalikööri,0.240964,0.289855,0.048891
1,France restauration,La corne d'abondance,Lakkalikööri,0.289855,0.240964,0.048891
2,Trail's Head Gourmet Provisioners,Bólido Comidas preparadas,Rhönbräu Klosterbier,0.224719,0.263158,0.038439
3,Bólido Comidas preparadas,Trail's Head Gourmet Provisioners,Rhönbräu Klosterbier,0.263158,0.224719,0.038439
4,GROSELLA-Restaurante,Bólido Comidas preparadas,Rhönbräu Klosterbier,0.294118,0.263158,0.030960
5,Bólido Comidas preparadas,GROSELLA-Restaurante,Rhönbräu Klosterbier,0.263158,0.294118,0.030960
6,Spécialités du monde,Bólido Comidas preparadas,Rhönbräu Klosterbier,0.291667,0.263158,0.028509
7,Bólido Comidas preparadas,Spécialités du monde,Rhönbräu Klosterbier,0.263158,0.291667,0.028509
8,Océano Atlántico Ltda.,Antonio Moreno Taquería,Queso Cabrales,0.227273,0.206128,0.021145
9,Antonio Moreno Taquería,Océano Atlántico Ltda.,Queso Cabrales,0.206128,0.227273,0.021145


# Collaborative filtering where we recommend based on customer similarity

In [263]:
query="""WITH 1 as neighbours
MATCH (me:customer)-[:similarity]->(c:customer)-[r:rated]->(p:product)
WHERE NOT ( (me)-[:rated]->(p:Product) )
WITH  p, COLLECT(r.rating)[0..neighbours] as ratings, collect(c.customer_name)[0..neighbours] as customers
WITH  p, customers, REDUCE(s=0,i in ratings | s+i) / SIZE(ratings)  as recommendation
WHERE recommendation > .15 AND recommendation < .3
RETURN p.prod_name, customers, recommendation 
ORDER BY recommendation DESC"""
n4j_helper.neo4j_run_query_pandas(query)

,p.prod_name,customers,recommendation
0,Escargots de Bourgogne,[Alfreds Futterkiste],0.229885
1,Gorgonzola Telino,[Around the Horn],0.176923
2,Sir Rodney's Scones,[Blauer See Delikatessen],0.164286
3,Konbu,[Ana Trujillo Emparedados y helados],0.158730
4,Mascarpone Fabioli,[Ana Trujillo Emparedados y helados],0.158730
5,Camembert Pierrot,[Ana Trujillo Emparedados y helados],0.158730
6,Mozzarella di Giovanni,[Ana Trujillo Emparedados y helados],0.158730


In [221]:
query="""WITH 1 as neighbours
MATCH (me:customer)-[:similarity]->(c:customer)-[r:rated]->(p:product)
WHERE NOT ( (me)-[:rated]->(p:Product) )
WITH  p, COLLECT(r.rating)[0..neighbours] as ratings, collect(c.customer_name)[0..neighbours] as customers
WITH  p, customers, REDUCE(s=0,i in ratings | s+i) / SIZE(ratings)  as recommendation
WHERE recommendation > .15 AND recommendation < .3
RETURN p.prod_name, customers, recommendation 
ORDER BY recommendation DESC"""

,p.prod_name,customers,recommendation
0,Louisiana Fiery Hot Pepper Sauce,[Furia Bacalhau e Frutos do Mar],0.157593
1,Chartreuse verte,[Familia Arquibaldo],0.140056
2,Uncle Bob's Organic Dried Pears,[Reggiani Caseifici],0.119403
3,Sir Rodney's Marmalade,[Océano Atlántico Ltda.],0.113636
4,Maxilaku,[Comércio Mineiro],0.112782
...,...,...,...
62,Genen Shouyu,[Save-a-lot Markets],0.010085
63,Ipoh Coffee,[Furia Bacalhau e Frutos do Mar],0.008596
64,Laughing Lumberjack Lager,[Lehmanns Marktstand],0.006297
65,Ravioli Angelo,[Save-a-lot Markets],0.003025


In [264]:
query = """MATCH (p1:product {prod_id: "prod_3"})<-[:rated]-(c:customer)-[:rated]->(p2:product)
WITH   p1.prod_name AS original_product, 
       p2.prod_name AS recommendation, 
       COUNT(*) AS customers_also_bought_n_times
WHERE  customers_also_bought_n_times > 7
RETURN original_product, recommendation, customers_also_bought_n_times
ORDER BY customers_also_bought_n_times DESC """
n4j_helper.neo4j_run_query_pandas(query)

,original_product,recommendation,customers_also_bought_n_times
0,Aniseed Syrup,Raclette Courdavault,9
1,Aniseed Syrup,Boston Crab Meat,8


# Customers who bought Aniseed Syrup , also bought that other products more then 5 times.

In [266]:
query = """
MATCH (p1:product {prod_id: "prod_3"})<-[:rated]-(c:customer)-[:rated]->(p2:product)
WITH   p1.prod_name AS original_product, 
       p2.prod_name AS recommendation, 
       COUNT(*) AS customers_also_bought_n_times
WHERE  customers_also_bought_n_times > 5
RETURN original_product, recommendation, customers_also_bought_n_times
ORDER BY customers_also_bought_n_times DESC 
"""
n4j_helper.neo4j_run_query_pandas(query)

,original_product,recommendation,customers_also_bought_n_times
0,Aniseed Syrup,Raclette Courdavault,9
1,Aniseed Syrup,Boston Crab Meat,8
2,Aniseed Syrup,Mozzarella di Giovanni,7
3,Aniseed Syrup,Tarte au sucre,7
4,Aniseed Syrup,Pâté chinois,7
5,Aniseed Syrup,Chartreuse verte,7
6,Aniseed Syrup,Uncle Bob's Organic Dried Pears,6
7,Aniseed Syrup,Ipoh Coffee,6
8,Aniseed Syrup,Camembert Pierrot,6
9,Aniseed Syrup,Jack's New England Clam Chowder,6


# Rating count for each product

In [267]:
query = """
MATCH (p:product)<-[:rated]-(c:customer)
WITH p.prod_name AS product_name, COUNT(*) AS rating_count
RETURN product_name, rating_count
ORDER BY rating_count DESC
"""
n4j_helper.neo4j_run_query_pandas(query)

,product_name,rating_count
0,Raclette Courdavault,43
1,Camembert Pierrot,36
2,Gorgonzola Telino,34
3,Jack's New England Clam Chowder,34
4,Gnocchi di nonna Alice,34
...,...,...
62,Valkoinen suklaa,9
63,Louisiana Hot Spiced Okra,8
64,Genen Shouyu,6
65,Gravad lax,6


# Category wise top rated products

In [268]:
query = """
MATCH (cat:ctg)<-[:part_of]-(p:product)<-[r:rated]-(c:customer)
WITH   cat.ctg_name AS category_name,
       p.prod_name AS product_name, 
       COUNT(*) AS rating_count
RETURN   category_name,
         product_name, 
         rating_count
ORDER BY category_name, 
         rating_count DESC
"""
n4j_helper.neo4j_run_query_pandas(query)

,category_name,product_name,rating_count
0,Beverages,Lakkalikööri,33
1,Beverages,Rhönbräu Klosterbier,33
2,Beverages,Outback Lager,30
3,Beverages,Steeleye Stout,28
4,Beverages,Ipoh Coffee,26
...,...,...,...
62,Seafood,Carnarvon Tigers,22
63,Seafood,Escargots de Bourgogne,17
64,Seafood,Rogede sild,12
65,Seafood,Röd Kaviar,11


# Creating prod_rating nodes

In [269]:
query = """
MATCH (cat:ctg)<-[:part_of]-(p:product)<-[r:rated]-(c:customer)
WITH   cat.ctg_name AS category_name,
       cat.ctg_id   AS category_id,
       p.prod_name  AS product_name,
       p.prod_id    AS product_id,
       COUNT(*) AS rating_count
RETURN   category_name,
         category_id,
         product_name, 
         product_id,
         rating_count
ORDER BY category_name, 
         rating_count DESC
"""
ctg_top_rated_prod_df = n4j_helper.neo4j_run_query_pandas(query)

for ctg_prod in ctg_top_rated_prod_df.values.tolist():
    ctg_id = ctg_prod[1]
    prod_id = ctg_prod[3]
    rating_count = ctg_prod[-1]
    label_name    = 'prod_rating'
    label_value   = prod_id
    label_details = f"{label_value}:{label_name}"
    node_property_string = '{prod_id: "' + prod_id + '", ctg_id: "' + ctg_id + '", rating_count: ' + str(rating_count) + '}'
    n4j_helper.neo4j_create_node(label_details   = label_details
                               , node_property   = node_property_string)
n4j_helper.neo4j_number_nodes_relationships()

ctg_top_rated_prod_df

-------------------------
  Nodes: 3245
  Relationships: 10033
-------------------------


,category_name,category_id,product_name,product_id,rating_count
0,Beverages,ctg_1,Lakkalikööri,prod_76,33
1,Beverages,ctg_1,Rhönbräu Klosterbier,prod_75,33
2,Beverages,ctg_1,Outback Lager,prod_70,30
3,Beverages,ctg_1,Steeleye Stout,prod_35,28
4,Beverages,ctg_1,Ipoh Coffee,prod_43,26
...,...,...,...,...,...
62,Seafood,ctg_8,Carnarvon Tigers,prod_18,22
63,Seafood,ctg_8,Escargots de Bourgogne,prod_58,17
64,Seafood,ctg_8,Rogede sild,prod_45,12
65,Seafood,ctg_8,Röd Kaviar,prod_73,11


# Create relationship between product and prod_rating

# (prod)-[:has_number_of]->(prod_rating)

In [270]:
query = """MATCH (p:product),(pr:prod_rating)
WHERE      p.prod_id = pr.prod_id
AND        p.ctg_id  = pr.ctg_id
MERGE      (p)-[:has_number_of]->(pr)"""
n4j_helper.neo4j_session.run(query)
n4j_helper.neo4j_number_nodes_relationships()

-------------------------
  Nodes: 3245
  Relationships: 10033
-------------------------


# Suggesting customers other top rated products in the same category which they have not bought


In [271]:
query = """MATCH (cust1:customer)-[r1:rated]->(p1:product)-[:part_of]->(cat:ctg)
MATCH (p2)-[:part_of]->(cat)
MATCH (p1)-[prt1:has_number_of]->(pr1:prod_rating)
MATCH (p2)-[prt2:has_number_of]->(pr2:prod_rating)
WHERE  NOT EXISTS ((cust1)-[r1:rated]->(p2))
AND    pr2.rating_count > pr1.rating_count
AND    cust1.customer_name = 'Around the Horn'
AND    cat.ctg_name        = "Beverages"
RETURN 
DISTINCT cust1.customer_name          AS customer_name
       , cat.ctg_name                 AS category_name              
       , p2.prod_name                 AS recommended_product
ORDER BY customer_name
       , category_name
       , recommended_product
"""
n4j_helper.neo4j_run_query_pandas(query)

,customer_name,category_name,recommended_product
0,Around the Horn,Beverages,Chartreuse verte
1,Around the Horn,Beverages,Côte de Blaye
2,Around the Horn,Beverages,Ipoh Coffee
3,Around the Horn,Beverages,Lakkalikööri
4,Around the Horn,Beverages,Outback Lager
5,Around the Horn,Beverages,Rhönbräu Klosterbier
6,Around the Horn,Beverages,Sasquatch Ale
7,Around the Horn,Beverages,Steeleye Stout
